In [28]:
import pandas as pd
import re
from rapidfuzz import process, fuzz
import numpy as np

# Load your dataset
df = pd.read_csv('/Users/yangyungchyi/Documents/Spring 2025/MIS 462/0420_1.csv', dtype=str)

# Replace actual NaN values with empty strings
df = df.replace(np.nan, "", regex=True)

# Replace string "nan" (in case it was converted earlier) with empty strings
df = df.replace("nan", "", regex=False)

# List of valid city names
valid_cities = [
    "Alexandria", "Annandale", "Arlington", "Baileys Crossroads", "Burke", "Centreville",
    "Chantilly", "Clifton", "Dunn Loring", "Falls Church", "Fairfax",
    "Fairfax Station", "Fort Belvoir", "Franconia", "Great Falls",
    "Herndon", "Kingstowne", "Leesburg", "Lincolnia", "Lorton", "Manassas", "McLean",
    "Merrifield", "Mount Vernon", "Oak Hill", "Oakton", "Reston",
    "Seven Corners", "South Riding", "Springfield", "Sterling", "Tysons Corner",
    "Vienna", "West McLean", "Woodbridge"
]

# --- Step 1: Address and City Cleanup ---
def clean_row(row):
    street1 = str(row.get("Street1", "") or "")
    street2 = str(row.get("Street2", "") or "")
    city = str(row.get("City", "") or "")
    building_name = str(row.get("BuildingName", "") or "")  # <-- keep existing building name if present

    street_keywords = r"\b(Rd|St|Blvd|Bvld|Bld|Blve|Center|Palaza|Way|Wy|Dr|Hwy|Ave|Ln|Ct|Cir|Pl|Tnpk|Glebe|Ctr|Pkwy|Parkway|Plaza|Trl|Route|Rt|Ter|Sq|Square|Park|Highway|Terrace|Trail|Plz|Lane|Way|Court|Circle|Pike|Drive|Road|Street|Cluster|Pk|Terr|Boulevard|Avenue|Jr|Highway|Heights|Rdg|Ridge|Turnpike|Tpke|Place)\b"
    suite_keywords = r"\b(Suite \d+|Ste \d+|Post Office Box \d+)\b"

    if re.search(r"C/O|Att:|Co\.", street1, re.IGNORECASE):
        building_name = street1
        if street2.strip():
            street1 = street2
            street2 = ""

    elif not re.search(street_keywords, street1, re.IGNORECASE) and re.search(street_keywords, street2, re.IGNORECASE):
        building_name = street1
        street1 = street2
        street2 = ""

    suite_match = re.search(suite_keywords, street1, re.IGNORECASE)
    if suite_match:
        street2 = suite_match.group(0)
        street1 = re.sub(suite_keywords, "", street1).strip()

    if not re.search(street_keywords, street1, re.IGNORECASE) and not building_name:
        building_name = street1
        street1 = ""

    suite_in_building = re.search(suite_keywords, building_name, re.IGNORECASE)
    if suite_in_building:
        if not street2.strip():
            street2 = suite_in_building.group(0)
        building_name = re.sub(suite_keywords, "", building_name).strip()

    # Final check: Only shift building_name if it's not already populated
    if not street1.strip() and building_name.strip():
        if not re.search(street_keywords, building_name, re.IGNORECASE):  # Extra safety
            street1 = building_name
            building_name = ""

    return pd.Series([street1.strip(), street2.strip(), building_name.strip(), city])


df[["Street1", "Street2", "BuildingName", "City"]] = df.apply(clean_row, axis=1)

# --- Step 2: Fuzzy match city without saving cleaned version ---
def clean_city(c):
    if pd.isna(c):
        return ""
    c = str(c).strip().upper()
    c = re.sub(r"[^\w\s]", "", c)
    c = re.sub(r"\s+", " ", c)
    return c

def is_valid_format(c):
    return not any([
        c == "",
        c.isdigit(),
        any(x in c for x in ["SUITE", "FLOOR", "PLACE", "UNIT", "BUILDING", "STREET", "DRIVE", "LANE", "BLVD"]),
        len(c.split()) > 4
    ])

def match_city(city):
    cleaned = clean_city(city)
    if is_valid_format(cleaned):
        match = process.extractOne(cleaned.title(), valid_cities, scorer=fuzz.WRatio)
        if match and match[1] >= 75:
            return match[0]
    return None

df["City"] = df["City"].apply(match_city)

# Clean up NaN values before address formatting
df["Street1"] = df["Street1"].fillna("")
df["Street2"] = df["Street2"].fillna("")
df["City"] = df["City"].fillna("")
df["State"] = df["State"].fillna("")
df["ZIPCODE"] = df["ZIPCODE"].fillna("")


# --- Step 3: Format full address using corrected city ---
def format_address(row):
    address_parts = [str(row['Street1']), str(row['Street2']) if pd.notna(row['Street2']) else '',
                     str(row['City']), str(row['State']), str(row['ZIPCODE'])]
    return ', '.join([part.strip() for part in address_parts if part.strip()])

df["Full_Address"] = df.apply(format_address, axis=1)

# Keep existing order and just move Full_Address if needed
if "Full_Address" in df.columns:
    cols = df.columns.tolist()
    cols.insert(cols.index("ZIPCODE") + 1, cols.pop(cols.index("Full_Address")))
    df = df[cols]
    
# Keep existing order and just move Building Name if needed
if "BuildingName" in df.columns:
    cols = df.columns.tolist()
    cols.insert(cols.index("IncorpState") + 1, cols.pop(cols.index("BuildingName")))
    df = df[cols]

# Replace NaNs with empty strings
df = df.fillna("").astype(str)

def format_address(row):
    address_parts = [str(row['Street1']), str(row['Street2']) if pd.notna(row['Street2']) else '',
                     str(row['City']), str(row['State']), str(row['ZIPCODE'])]
    return ', '.join([part.strip() for part in address_parts if part.strip()])

df["Full_Address"] = df.apply(format_address, axis=1)


# Save to CSV
df.to_csv('0420_2.csv', index=False)
